In [8]:
import requests # A biblioteca requests é utilizada para fazer solicitações HTTP em Python. Ela permite que você envie solicitações HTTP/1.1 de forma fácil e intuitiva, sem a necessidade de adicionar manualmente consultas aos seus URLs ou codificar seus dados POST.
import folium # folium é uma biblioteca em Python que facilita a visualização de dados manipulados em Python em mapas interativos. Essencialmente, ela pode ajudar a transformar qualquer dado geoespacial em um mapa visualizável. Com ela, você pode criar vários tipos de mapas, adicionar marcadores, camadas, polígonos, entre outros.
import polyline # A biblioteca polyline é usada para codificar e decodificar polilinhas conforme a documentação do Google Maps. Uma polilinha é uma lista de pontos, onde cada ponto é sua própria lista de coordenadas. Essa codificação permite que a série de coordenadas seja convertida em uma string compacta, que pode ser facilmente transmitida ou armazenada, e depois revertida de volta a uma lista de coordenadas.
import numpy as np # numpy (frequentemente abreviado como np) é uma biblioteca para a linguagem Python que suporta grandes arrays e matrizes multidimensionais, além de uma vasta coleção de funções matemáticas de alto nível para operar nesses arrays. Ela é essencial para operações matemáticas e científicas em Python, fornecendo suporte para operações como multiplicação de matrizes, transformadas de Fourier, e soluções de equações diferenciais.
import pandas as pd # pandas é uma biblioteca em Python que fornece estruturas de dados e funções necessárias para a limpeza e análise rápida de dados. É frequentemente abreviada como pd quando importada. As estruturas de dados primárias em pandas são Series e DataFrame. DataFrame é essencialmente uma tabela que pode armazenar e manipular dados tabulares (como planilhas do Excel ou tabelas SQL), enquanto Series é uma única coluna de um DataFrame.

In [9]:
def get_route(pickup_lon, pickup_lat, dropoff_lon, dropoff_lat):
    
    # Formatando as coordenadas de coleta e entrega em uma única string.
    loc = "{},{};{},{}".format(pickup_lon, pickup_lat, dropoff_lon, dropoff_lat)
    
    # Definindo a URL base do serviço OSRM que está rodando localmente.
    url = "http://127.0.0.1:5000/route/v1/driving/"
    
    # Realizando a solicitação GET para obter os detalhes da rota.
    r = requests.get(url + loc)
    
    # Verificando se a solicitação foi bem-sucedida.
    if r.status_code != 200:
        return {}
    
    # Convertendo a resposta em formato JSON.
    res = r.json()
    
    # Decodificando a polilinha da rota para obter as coordenadas.
    routes = polyline.decode(res['routes'][0]['geometry'])
    
    # Extraindo as coordenadas do ponto de partida.
    start_point = [res['waypoints'][0]['location'][1], res['waypoints'][0]['location'][0]]
    
    # Extraindo as coordenadas do ponto de entrega.
    end_point = [res['waypoints'][1]['location'][1], res['waypoints'][1]['location'][0]]
    
    # Extraindo a distância total da rota.
    distance = res['routes'][0]['distance']
    
    # Compilando os dados extraídos em um dicionário.
    out = {
        'route': routes,
        'start_point': start_point,
        'end_point': end_point,
        'distance': distance
    }
    
    # Retornando o dicionário com os detalhes da rota.
    return out

In [10]:
# Definindo as coordenadas de partida e entrega.
# pickup_lon e pickup_lat são as coordenadas de longitude e latitude do ponto de partida, respectivamente.
# dropoff_lon e dropoff_lat são as coordenadas de longitude e latitude do ponto de entrega, respectivamente.
pickup_lon, pickup_lat, dropoff_lon, dropoff_lat = -47.870891, -15.764467, -47.874652, -15.817366

# Chamando a função 'get_route' com as coordenadas definidas acima.
# A função irá consultar a rota entre os pontos de partida e entrega e retornará os detalhes da rota.
test_route = get_route(pickup_lon, pickup_lat, dropoff_lon, dropoff_lat)

# Exibindo o retorno da função 'get_route'. Isso mostrará os detalhes da rota, incluindo as coordenadas da rota,
# o ponto de partida, o ponto de entrega e a distância total da rota.
test_route

{'route': [(-15.76451, -47.87096),
  (-15.77091, -47.86642),
  (-15.7713, -47.86387),
  (-15.77427, -47.8645),
  (-15.77573, -47.86416),
  (-15.78431, -47.86068),
  (-15.78766, -47.85835),
  (-15.79164, -47.85884),
  (-15.7985, -47.85233),
  (-15.80219, -47.85089),
  (-15.80308, -47.85136),
  (-15.8089, -47.85991),
  (-15.81822, -47.87768),
  (-15.81738, -47.87465)],
 'start_point': [-15.764511, -47.870959],
 'end_point': [-15.817375, -47.874648],
 'distance': 8862.4}

In [11]:
# Definindo a função get_map que aceita um dicionário de rotas como entrada.
def get_map(route):

    # Iniciando um mapa do folium.
    # A localização central do mapa é definida como a média das latitudes e longitudes dos pontos de partida e chegada.
    # Isso garante que o mapa seja centralizado entre os pontos de partida e chegada.
    m = folium.Map(location=[(route['start_point'][0] + route['end_point'][0])/2,
                             (route['start_point'][1] + route['end_point'][1])/2],
                   zoom_start=13)  # O zoom inicial é definido para 13.

    # Adicionando um PolyLine ao mapa. 
    # Este PolyLine representa a rota entre os pontos de partida e chegada.
    folium.PolyLine(
        route['route'],         # Lista de coordenadas representando a rota.
        weight=8,               # Largura da linha da rota.
        color='blue',           # Cor da linha da rota.
        opacity=0.6             # Opacidade da linha da rota.
    ).add_to(m)                # Adicionando o PolyLine ao mapa.

    # Adicionando um marcador ao mapa para o ponto de partida.
    folium.Marker(
        location=route['start_point'],             # Localização do marcador.
        icon=folium.Icon(icon='play', color='green')  # Usando um ícone de "play" de cor verde para representar o ponto de partida.
    ).add_to(m)                                   # Adicionando o marcador ao mapa.

    # Adicionando um marcador ao mapa para o ponto de entrega.
    folium.Marker(
        location=route['end_point'],               # Localização do marcador.
        icon=folium.Icon(icon='stop', color='red')     # Usando um ícone de "stop" de cor vermelha para representar o ponto de chegada.
    ).add_to(m)                                       # Adicionando o marcador ao mapa.

    # Retornando o mapa com a rota e os marcadores.
    return m


In [12]:
get_map(test_route)